In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt
from scipy.stats.mstats import gmean
import numpy as np
import re

In [ ]:
# Output folders.

plot_dir = Path('./plots')
table_dir = Path('./tables')

make_plots = False
make_tables = True

if make_plots:
    plot_dir.mkdir(parents=True, exist_ok=True)
if make_tables:
    table_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
configs = {'LP-2fu-nofuse':'LP-NF-2', 'LP-2fu-fuse':'LP-F-2',
           'LP-1fu-nofuse':'LP-NF-1', 'LP-1fu-fuse':'LP-F-1',
           'HP-3fu-nofuse':'HP-NF-3', 'HP-3fu-fuse':'HP-F-3',
           'HP-2fu-nofuse':'HP-NF-2', 'HP-2fu-fuse':'HP-F-2',  
           'HP-1fu-nofuse':'HP-NF-1', 'HP-1fu-fuse':'HP-F-1'}
configs_order = ['LP-NF-2', 'LP-F-2', 'LP-NF-1', 'LP-F-1',
                 'HP-NF-3', 'HP-F-3', 'HP-NF-2', 'HP-F-2', 'HP-NF-1', 'HP-F-1']

In [ ]:
benchmarks = {
    #'hmmer-small':'hmmer',
    #'libquantum-small':'libquantum',
    #'blackscholes_simsmall_1':'blackscholes',
    #'freqmine_simsmall_1':'freqmine',
    #'swaptions_simsmall_1':'swaptions',
    #'barnes_simsmall_1':'barnes',
    #'cholesky_simsmall_1':'cholesky',
    #'fft_simsmall_1':'fft-splash',
    #'fmm_simsmall_1':'fmm',
    #'ocean_cp_simsmall_1':'ocean',
    #'radix_simsmall_1':'radix',
    #'raytrace_simsmall_1':'raytrace',
    #'water_nsquared_simsmall_1':'water-nsquared',
    #'water_spatial_simsmall_1':'water-spatial',
    'integerNN_1':'intNN',
    'streamvbyte_1':'streamvbyte',
    'img_cartoon_tiny_image1_50per':'cartoon',
    'img_cartoon_tiny_image2_50per':'cartoon',
    'img_cartoon_tiny_image3_50per':'cartoon',
    'img_cartoon_tiny_image4_50per':'cartoon',
    'img_cartoon_tiny_image5_50per':'cartoon',
    'img_canny_tiny_image1_50per':'canny',
    'img_canny_tiny_image2_50per':'canny',
    'img_canny_tiny_image3_50per':'canny',
    'img_canny_tiny_image4_50per':'canny',
    'img_canny_tiny_image5_50per':'canny',
    'img_hist_tiny_image1_50per':'hist',
    'img_hist_tiny_image2_50per':'hist',
    'img_hist_tiny_image3_50per':'hist',
    'img_hist_tiny_image4_50per':'hist',
    'img_hist_tiny_image5_50per':'hist',
    'img_integral_tiny_image1_50per':'img_integral',
    'img_integral_tiny_image2_50per':'img_integral',
    'img_integral_tiny_image3_50per':'img_integral',
    'img_integral_tiny_image4_50per':'img_integral',
    'img_integral_tiny_image5_50per':'img_integral',
    'conv_tiny_image1_50per':'conv',
    'conv_tiny_image2_50per':'conv',
    'conv_tiny_image3_50per':'conv',
    'conv_tiny_image4_50per':'conv',
    'conv_tiny_image5_50per':'conv',
    'img_erode_tiny_image1_50per':'erode',
    'img_erode_tiny_image2_50per':'erode',
    'img_erode_tiny_image3_50per':'erode',
    'img_erode_tiny_image4_50per':'erode',
    'img_erode_tiny_image5_50per':'erode',
    'img_median_tiny_image1_50per':'median',
    'img_median_tiny_image2_50per':'median',
    'img_median_tiny_image3_50per':'median',
    'img_median_tiny_image4_50per':'median',
    'img_median_tiny_image5_50per':'median',
    'amax_cols_tiny_16bit_0':'amax',
    'amax_cols_tiny_16bit_1':'amax',
    'amax_cols_tiny_16bit_2':'amax',
    'amax_cols_tiny_16bit_3':'amax',
    'amax_cols_tiny_16bit_4':'amax',
    'asum_cols_tiny_16bit_0':'asum',
    'asum_cols_tiny_16bit_1':'asum',
    'asum_cols_tiny_16bit_2':'asum',
    'asum_cols_tiny_16bit_3':'asum',
    'asum_cols_tiny_16bit_4':'asum',
    'gemv_tiny_16bit_0':'gemv',
    'gemv_tiny_16bit_1':'gemv',
    'gemv_tiny_16bit_2':'gemv',
    'gemv_tiny_16bit_3':'gemv',
    'gemv_tiny_16bit_4':'gemv',
    'ger_tiny_8bit_0':'ger',
    'ger_tiny_8bit_1':'ger',
    'ger_tiny_8bit_2':'ger',
    'ger_tiny_8bit_3':'ger',
    'ger_tiny_8bit_4':'ger',
    'sqnrm2_cols_tiny_16bit_0':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_1':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_2':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_3':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_4':'sqnrm2',
    'gemm_tiny_16bit_0':'gemm',
    'gemm_tiny_16bit_1':'gemm',
    'gemm_tiny_16bit_2':'gemm',
    'gemm_tiny_16bit_3':'gemm',
    'gemm_tiny_16bit_4':'gemm',
    #'fft_tiny_16bit_0':'fft',
    #'fft_tiny_16bit_1':'fft',
    #'fft_tiny_16bit_2':'fft',
    #'fft_tiny_16bit_3':'fft',
    #'fft_tiny_16bit_4':'fft',
    'fft_tiny_12bit_0':'fft',
    'fft_tiny_12bit_1':'fft',
    'fft_tiny_12bit_2':'fft',
    'fft_tiny_12bit_3':'fft',
    'fft_tiny_12bit_4':'fft',
    #'fft_tiny_8bit_0':'fft',
    #'fft_tiny_8bit_1':'fft',
    #'fft_tiny_8bit_2':'fft',
    #'fft_tiny_8bit_3':'fft',
    #'fft_tiny_8bit_4':'fft'
}

bench_order = ['hmmer', 'streamvbyte', 'integerNN', 'cartoon', 'sqnrm2', 'amax', 'gemm', 'fft', 'conv', 'median', 'img_integral', 'canny', 'erode']
bench_row = {'hmmer':1, 'streamvbyte':1, 'integerNN':1, 'cartoon':1, 'sqnrm2':1, 'amax':1, 'gemm':1, 'fft':2, 'conv':2, 'median':2, 'img_integral':2, 'canny':2, 'erode':2}

In [ ]:
stats = Path('./stats/')
stats = stats.glob('**/roi.txt')
stats = list(stats)
stats = [p.parents[0] for p in stats]
#stats

In [ ]:
df = []

for s in stats:
    bench  = s.name
    config = s.parents[1].name
    
    if bench in benchmarks.keys() and config in configs.keys():
        try:
            usage = pd.read_csv(s/'simd_fu_used.csv')
        except EmptyDataError:
            continue

        usage = usage.drop(columns=['samples', 'mean', 'stdev', 'underflows', 'overflows', 'min_value', 'max_value'])
        usage = 100 * usage / int(usage['total'])
        usage = usage.drop(columns=['0', 'total'])
        usage['Benchmark'] = bench
        usage['Config'] = config
        usage = usage.melt(id_vars=['Benchmark', 'Config'], var_name='Var', value_name='Percentage')

        df.append(usage)
        
df = pd.concat(df)
df['Config'] = df['Config'].replace(configs)
df = df.set_index(['Config', 'Benchmark', 'Var'])
df = df.groupby(by=[None, benchmarks, None], level=['Config', 'Benchmark', 'Var']).mean()
df = df.reset_index()
df['Row'] = df['Benchmark'].replace(bench_row)
df

In [ ]:
usage_plot = alt.Chart(df).mark_bar(size=16).encode(
    y=alt.Y(
        'Percentage:Q',
        scale=alt.Scale(domain=(0, 100)),
        title="Percentage of execution cycles [%]"
    ),
    x=alt.X(
        'Config:N',
        sort=configs_order,
        title=""
    ),
    column=alt.Column(
        'Benchmark:N',
        sort=bench_order,
        spacing=8
    ),
    color=alt.Color(
        'Var:O',
        scale=alt.Scale(scheme='yellowgreenblue'),
        title='Number of active SIMD units'
    ),
).properties(
    height=alt.Step(19)
).facet(
    row=alt.Row(
        'Row:O',
        title=""
    )
).configure_axis(
    labelFontSize=17, titleFontSize=20
).configure_header(
    labelFontSize=21, titleFontSize=21
).configure_legend(
    titleFontSize=21, labelFontSize=21, orient='top', titleLimit=0
)
usage_plot

In [ ]:
#usage_plot.save('plots/simd_usage.svg', webdriver='firefox')